In [1]:
# from transformers import AutoTokenizer
# from model import RegBertForQA  
# from ExplanationGenerator import Generator 
# from captum.attr import visualization
# from BERT import BertModel
# import torch

# model = RegBertForQA.from_pretrained("fine_tuned_model_orig").to("cuda")

# # model = RegBertForQA.from_pretrained("fine_tuned_model_registers_Nov17").to("cuda")
# model.eval()

# tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model_orig")
# # tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model_registers_Nov17")
# explanations = Generator(model)

# question, context = "What is a way to increase your wound healing speed?", \
# "Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care."

# # context = "Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1"
# # question = "When was quantum field theory developed?"
# # num_registers = 50
# # register_token_ids = ['[REG{}]'.format(i) for i in range(num_registers)]
# # input_tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))
# # all_tokens =  register_token_ids + input_tokens
# encoding = tokenizer(
#     question,
#     context,
#     return_tensors="pt",
#     # padding="max_length",
#     # truncation=True,
#     max_length=434
# )
# input_ids = encoding["input_ids"].to("cuda")
# # token_type_ids = encoding['token_type_ids'].to("cuda")  
# attention_mask = encoding["attention_mask"].to("cuda")

# expl = explanations.generate_LRP(
#     input_ids=input_ids, 
#     attention_mask=attention_mask, 
#     # start_positions=None,  
#     # end_positions=None,   
#     start_layer=0
# )[0]
from transformers import AutoTokenizer
from model import RegBertForQA  
from ExplanationGenerator import Generator 
from captum.attr import visualization
from BERT import BertModel
import torch

model = RegBertForQA.from_pretrained("fine_tuned_model_orig").to("cuda")
# model = RegBertForQA.from_pretrained("fine_tuned_model_registers_Nov17").to("cuda")
model.eval()

tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model_orig")
# tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model_registers_Nov17")
explanations = Generator(model)

question, context = "What is a way to increase your wound healing speed?", \
"Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care."

# num_registers = 50
# register_token_ids = ['[REG{}]'.format(i) for i in range(num_registers)]
# input_tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))
# all_tokens =  register_token_ids + input_tokens
encoding = tokenizer(
    question,
    context,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=434
)
input_ids = encoding["input_ids"].to("cuda")
attention_mask = encoding["attention_mask"].to("cuda")

expl = explanations.generate_LRP(
    input_ids=input_ids, 
    attention_mask=attention_mask, 
    # start_positions=None,  
    # end_positions=None,   
    start_layer=0
)[0]

/home/gnilay/.conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gnilay/.conda/envs/llm/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/gnilay/.conda/envs/llm/lib/python3.10/site-packages/torc

from regbertfor QA, num_reg= 50


Some weights of RegBert were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.position_ids', 'bert.reg_pos', 'bert.reg_tokens']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RegBertForQA were not initialized from the model checkpoint at fine_tuned_model_orig and are newly initialized: ['bert.embeddings.position_ids', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'bert.reg_pos', 'bert.reg_tokens']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: zeros() received an invalid combination of arguments - got (torch.Size, dev=str, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
# adjusted_start_index, adjusted_end_index
# ! pip install captum

In [ ]:
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
# print(model._logits[0], len(model._logits))
# output = torch.nn.functional.softmax(model._logits[0], dim=-1)
# print(output.shape)
# classification = output.argmax(dim=-1).item()
# # get class name
# class_name = classifications[classification]
# # if the classification is negative, higher explanation scores are more negative
# # flip for visualization
# if class_name == "NEGATIVE":
#   expl *= (-1)

# tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
# print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
# vis_data_records = [visualization.VisualizationDataRecord(
#                                 expl,
#                                 output[0][classification],
#                                 classification,
#                                 true_class,
#                                 true_class,
#                                 1,       
#                                 tokens,
#                                 1)]
# visualization.visualize_text(vis_data_records)
outputs = model(
    input_ids=input_ids, attention_mask=attention_mask
)
num_registers = model.bert.num_registers
start_logits = outputs.start_logits
end_logits = outputs.end_logits

start_probs = torch.softmax(start_logits, dim=-1)
end_probs = torch.softmax(end_logits, dim=-1)

start_index = torch.argmax(start_probs, dim=-1).item()
end_index = torch.argmax(end_probs, dim=-1).item()

adjusted_start_index = max(0, start_index - num_registers)
adjusted_end_index = max(0, end_index - num_registers)

pred_prob = ((start_probs[0, start_index] + end_probs[0, end_index]) / 2).item()

answer_ids = input_ids[0, adjusted_start_index: adjusted_end_index + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
predicted_answer = tokenizer.convert_tokens_to_string(answer_tokens)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())

vis_data_record = visualization.VisualizationDataRecord(
    word_attributions=expl,
    pred_prob=pred_prob,
    pred_class=predicted_answer,
    true_class="",  # Provide true answer if available
    attr_class="",
    attr_score=1,
    raw_input_ids=all_tokens,
    convergence_score=1
)

# Visualize attributions
visualization.visualize_text([vis_data_record])


### With registers

In [ ]:
from transformers import AutoTokenizer
from model import RegBertForQA  
from ExplanationGenerator import Generator 
from captum.attr import visualization
from BERT import BertModel
import torch

# model = RegBertForQA.from_pretrained("fine_tuned_model_orig").to("cuda")
model = RegBertForQA.from_pretrained("fine_tuned_model_registers_Nov17").to("cuda")
model.eval()

# tokenizer = AutoTokenizer.from_pretrained("../fine_tuned_model_orig")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model_registers_Nov17")
explanations = Generator(model)

question, context = "What is a way to increase your wound healing speed?", \
"Wound care encourages and speeds wound healing via cleaning and protection from reinjury or infection. Depending on each patient's needs, it can range from the simplest first aid to entire nursing specialties such as wound, ostomy, and continence nursing and burn center care."

# num_registers = 50
# register_token_ids = ['[REG{}]'.format(i) for i in range(num_registers)]
# input_tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))
# all_tokens =  register_token_ids + input_tokens
encoding = tokenizer.encode_plus(
    question,
    context,
    return_tensors="pt",
    # padding="max_length",
    # truncation=True,
    # max_length=434
)
input_ids = encoding["input_ids"].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")  
attention_mask = encoding["attention_mask"].to("cuda")

expl = explanations.generate_LRP(
    input_ids=input_ids, 
    attention_mask=attention_mask, 
    # start_positions=None,  
    # end_positions=None,   
    start_layer=0
)[0]

In [ ]:
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

outputs = model(
    input_ids=input_ids, attention_mask=attention_mask,
    token_type_ids=token_type_ids,
)
num_registers = model.bert.num_registers
print(num_registers)
start_logits = outputs.start_logits
end_logits = outputs.end_logits

start_probs = torch.softmax(start_logits, dim=-1)
end_probs = torch.softmax(end_logits, dim=-1)

start_index = torch.argmax(start_probs, dim=-1).item()
end_index = torch.argmax(end_probs, dim=-1).item()

adjusted_start_index = max(0, start_index - num_registers)
adjusted_end_index = max(0, end_index - num_registers)

pred_prob = ((start_probs[0, start_index] + end_probs[0, end_index]) / 2).item()

answer_ids = input_ids[0, adjusted_start_index: adjusted_end_index + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids)
predicted_answer = tokenizer.convert_tokens_to_string(answer_tokens)

register_token_ids = ['[REG{}]'.format(i) for i in range(num_registers)]

input_tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))

all_tokens =  register_token_ids + input_tokens
all_tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())

vis_data_record = visualization.VisualizationDataRecord(
    word_attributions=expl,
    pred_prob=pred_prob,
    pred_class=predicted_answer,
    true_class="",  # Provide true answer if available
    attr_class="",
    attr_score=1,
    raw_input_ids=all_tokens,
    convergence_score=1
)
print(adjusted_start_index, adjusted_end_index)
# Visualize attributions
visualization.visualize_text([vis_data_record])


In [ ]:

# Normalize scores for visualization
expl_start = (expl_start - expl_start.min()) / (expl_start.max() - expl_start.min())
# expl_end = (expl_end - expl_end.min()) / (expl_end.max() - expl_end.min())

# Get the model's predicted start and end indices
output = model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
start_idx = torch.argmax(output.start_logits, dim=-1).item()
# end_idx = torch.argmax(output.end_logits, dim=-1).item()

# Decode the predicted answer
tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
# predicted_answer = tokenizer.decode(input_ids[0][start_idx:end_idx + 1])

# Display explanation scores and tokens
# print("Predicted Answer:", predicted_answer)
print("Start Token Explanations:")
print([(tokens[i], expl_start[i].item()) for i in range(len(tokens))])
# print("End Token Explanations:")
# print([(tokens[i], expl_end[i].item()) for i in range(len(tokens))])

# Visualize explanations for start and end
vis_data_records_start = [visualization.VisualizationDataRecord(
    expl_start,
    output.start_logits[0][start_idx].item(),
    start_idx,
    None,
    None,
    1,       
    tokens,
    1
)]
# vis_data_records_end = [visualization.VisualizationDataRecord(
#     expl_end,
#     output.end_logits[0][end_idx].item(),
#     end_idx,
#     None,
#     None,
#     1,       
#     tokens,
#     1
# )]

print("\nVisualizing Start Explanations:")
visualization.visualize_text(vis_data_records_start)

# print("\nVisualizing End Explanations:")
# visualization.visualize_text(vis_data_records_end)
